# Clase 1 - Clasificación
![picture](https://drive.google.com/uc?id=1Ar6KpqlsLxcLlnOrXPx6QZejStP6bV9P)

## Pre requisitos

Se actualiza fastai descargando y ejecutando el script *colab*

In [0]:
 !curl -s https://course.fast.ai/setup/colab | bash

Dar permiso para acceder a mi Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
gdrive_dir = "/content/gdrive/My Drive/"
curso_dir = gdrive_dir + 'Colab Notebooks/Curso fastai/'

Nos aseguramos que cualquier cambio en una librería es recargado automáticamente y que que cualquier gráfica o imagen se muestre aquí

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Curso 1- Clasificación

Importamos los módulos de las librerías que necesitamos

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate

Establecemos un tamaño de *batch size*, si obtenemos un error de *out of memory* reiniciar el kernel y usar un tamaño de *batch size* menor

In [0]:
bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

### Descarga, preparación y visualización de los datos

Vamos a usar la base de datos [Oxford-IIIT Pet Dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/) del paper [O. M. Parkhi et al., 2012](http://www.robots.ox.ac.uk/~vgg/publications/2012/parkhi12a/parkhi12a.pdf), la cual tiene etiquetadas 12 razas distintas de gatos y 25 de perros.
Vamos a crear un modelo que aprenda a distinguir entre las 37 razas distintas
Según el paper, en 2012 lo mejor que pudieron hacer fue un modelo que con una precisión del 59.21% clasificaba las razas.

Para descargarnos los datos usaremos la función *untar_data* que descarga y descomprime los datos de una dirección dada.
En la propia librería de fastai ya tenemos algunas direcciones guardadas en URLs

In [0]:
help(untar_data)

In [0]:
untar_data??

In [0]:
path = untar_data(URLs.PETS); path

Fastai provee la función ls() para variables de tipo path, por si queremos saber qué hay en un directorio

In [0]:
path.ls()

Como se puede ver hay dos carpetas, una con las imágenes y otra con las anotaciones, creamos dos variables de tipo path con cada una

In [0]:
path_anno = path/'annotations'
path_img = path/'images'

In [0]:
path_img.ls()

Como se puede ver, la raza de cada imagen está escrita en el propio nombre de la imagen, por lo que usaremos el método `ImageDataBunch.from_name_re` para preparar los datos. Este metod obtiene las etiquetas de los nombres a partir de [regular expression](https://docs.python.org/3.6/library/re.html).

Primero obtenemos los nombres de todas las imagenes

In [0]:
fnames = get_image_files(path_img)

# Se muestran los primero 5 nombres
fnames[:5]

A continuación creamos la expresión regular

In [0]:
pat = r'/([^/]+)_\d+.jpg$'

Con esta expresión regular se obtienen las etiquetas de cada imagen, a continuación se muestra cómo lo hace

In [0]:
for fname in fnames:
  regExp = re.findall(pat, str(fname))
  if regExp:
    print(str(fname), end='\t\t')
    print(str(regExp))

Creamos una semilla fija. 

Cuando se quiere entrenar una red neuronal se suelen hacer pruebas cambiando parámetros. Se van cambiando parámetros para intentar obtener mejores resultados, pero si las imágenes que se usan para entrenar la red neuronal no son siempre las mismas no podemos saber si cada distinto resultado es por el cambio en el parámetro que hayamos hecho, o porque las imágenes de entrenamiento son distintas

Nosotros vamos a coger todas las imágenes y unas las reservaremos para el entrenamiento y otras para validación, de modo que para que siempre sean las mismas se deja la semilla fija y siempre la misma, porque si no cada vez se elegirian unas distintas al azahar.

In [0]:
np.random.seed(2)

Aumento de datos

El aumento de datos es quizás la técnica de regularización más importante cuando se entrena un modelo para visión artificial: en lugar de alimentar al modelo con las mismas imágenes cada vez, hacemos pequeñas transformaciones aleatorias (un poco de rotación, zoom, traducción, etc.) que no cambie lo que hay dentro de la imagen (para el ojo humano), pero cambie sus valores de píxeles. Los modelos entrenados con aumento de datos se generalizarán mejor.

Para obtener un conjunto de transformaciones con valores predeterminados que funcionen bastante bien en una amplia gama de tareas, a menudo es más fácil de usar get_transforms

Size es el numero de pixeles de las imagenes

Se normalizan los datos segun imagenet

Se crea el dataset a partir de los nombres de las imagenes

In [0]:
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=224, bs=bs).normalize(imagenet_stats)

Vemos una pequeña muestra del dataset creado para comprobar que está todo bien

In [0]:
data.show_batch(rows=4, figsize=(10,10))

Vemos las clases ddl dataset para comprobar que está todo bien

In [0]:
print(data.classes)
len(data.classes),data.c

### Training: resnet34

Vamos a hacer transfer learning. Es decir, vamos a coger una red neuronal convolucional de tipo resnet34, ya entrenada con el dataset [imagenet](http://www.image-net.org/) y vamos a añadirle unas capas extra para que se ajuste a nuestro problema

Creamos nuestro modelo, le tenemos que decir cual va a ser nuestro dataset (data), la arquitectura de la red neuronal (models.resnet34) y la métrica que queremos usar (metrics=error_rate)

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

Podemos ver cómo es la arquitectura de nuestro modelo

In [0]:
learn.model

Entrenamos las últimas capas, las que se han creado nuevas para nuestro problema. Las entrenamos durante 4 épocas, es decir, usará el dataset 4 veces para entrenar el modelo

In [0]:
learn.fit_one_cycle(4)

Como se puede ver, con estos sencillos pasos, y con solo unos 6 minutos de entrenamiento, se ha conseguido una red que clasifica con un error pequeño, mucho mejor que los resultados obtenidos en el paper original

Guardamos el modelo obtenido

In [0]:
learn.save('stage-1')

### Resultados

Veamos qué resultados tenemos.

Primero veremos cuáles fueron las categorías que el modelo confundió más entre sí. Intentaremos ver si lo que pronosticó el modelo fue razonable o no.

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [0]:
interp.plot_top_losses(9, figsize=(15,15))

En este caso, los errores parecen razonables (ninguno de los errores parece obviamente ingenuo). Este es un indicador de que nuestro clasificador funciona correctamente.

In [0]:
doc(interp.plot_top_losses)

Además, cuando graficamos la matriz de confusión, podemos ver que la distribución está muy sesgada: el modelo comete los mismos errores una y otra vez, pero rara vez confunde otras categorías. Esto sugiere que simplemente le resulta difícil distinguir algunas categorías específicas entre sí; Este es un comportamiento normal.

In [0]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [0]:
interp.most_confused(min_val=2)

### Unfreezing, fine-tuning, y learning rates

Como digimos habíamos cogido una red neuronal ya entrenada (resnet34), añadimos unas pocas capas al final y las entrenamos. Esto está bien, ya que las últimas capas estan entrenadas para nuestro problema, pero el resto de pesos de las anteriores capas fueron entrenados para el problema de imagenet.
Es decir, la red estaba congelada y solo entrenamos las capas que añadimos al final

Ahora vamos a descongelar toda la red y reentrenarla. Esto supone mucho menos trabajo que haberla entrenado desde cero, ya que los pesos no deberían variar mucho de los valores que ya tienen

Descongelamos la red

In [0]:
learn.unfreeze()

Hacemos el entrenamiento de una época de la red

In [0]:
learn.fit_one_cycle(1)

Como vemos obtenemos ahora un error mayor, peor que cuando la red estaba congelada, puede parecer que esto ha sido peor...

In [0]:
help(fit_one_cycle)

In [0]:
fit_one_cycle??

Volvemos a cargar la red congelada y en la que entrenamos las últimas capas que añadimos

In [0]:
learn.load('stage-1');

Vamos a usar una herramienta que nos da fastai para encontrar el valor del learning rate que le viene mejor a la red

In [0]:
help(lr_find)

In [0]:
lr_find??

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

Como se puede ver en la info de ```fit_one_cicle()``` por defecto entrena con un learning rate de 3x10<sup>-3</sup>, sin embargo, como vemos en la gráfica anterior, con ese learning rate obtenemos un error mayor que con otros valores menores.

Por lo que entrenamos pero con otros valores



Ojo! Como volvimos a cargar el modelo hay que volver a descongelarlo

In [0]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

Ahora obtenemos un error menor que el que obtuvimos cuando la red estaba congelada

Vamos a guardar y exportar la red

In [0]:
learn.save('stage-2')

In [0]:
learn.export()

El modelo se guarda donde están todas las imagenes. Así que lo copiamos a la carpeta *models* para tenerlo más localizado

In [0]:
!cp /content/data/oxford-iiit-pet/images/export.pkl /content/models

Lo renombramos

In [0]:
!mv /content/models/export.pkl /content/models/fastai_01_resnet34.pkl

In [0]:
!ls /content/models

### Training: resnet50

Vamos a repetir los pasos pero con una arquitectura de resnet50, la arquitectura resnet34 corresponde a una red de 34 capas, mientras que la resnet50 es una red de 50 capas.

Al tener más capas (al ser más profunda) debería hacer que tengamos mejores resultados

Vamos a usar imágenes más grandes, lo que va a hacer que tengamos que disminuir el tamaño del batch size para no tener problemas de memoria en la GPU

In [0]:
# Data con resnet34
# data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=224, bs=bs).normalize(imagenet_stats)

In [0]:
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=299, bs=bs//2).normalize(imagenet_stats)

Creamos el modelo

In [0]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

Lo entrenamos sin descongelar

In [0]:
learn.fit_one_cycle(8)

Vemos que con la arquitectura resnet50 obtenemos un resultado mucho mejor

Lo guardamos

In [0]:
learn.save('stage-1-50')

Buscamos el mejor valor del learning rate

In [0]:
learn.lr_find()
learn.recorder.plot()

Descongelamos y lo entrenamos con el mejor valor posible del learning rate

In [0]:
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-6,1e-4))

Lo guardamos

In [0]:
learn.save('stage-2-50')

Lo exportamos y lo llevamos a la carpeta *models*

In [0]:
learn.export()

In [0]:
!cp /content/data/oxford-iiit-pet/images/export.pkl /content/models

In [0]:
!mv /content/models/export.pkl /content/models/fastai_01_resnet50.pkl

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
interp.most_confused(min_val=2)

### Descarga de los modelos

In [0]:
!cp /content/models/fastai_01_resnet34.pkl /content/gdrive/My\ Drive/Colab\ Notebooks/Curso\ fastai/01\ Clasificacion

In [0]:
!cp /content/models/fastai_01_resnet50.pkl /content/gdrive/My\ Drive/Colab\ Notebooks/Curso\ fastai/01\ Clasificacion